In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian
print(hamiltonian)


In [2]:
import numpy as np 
from qiskit_nature.second_q.operators import PolynomialTensor
from qiskit_nature.second_q.properties import ElectronicDipoleMoment


dipole: ElectronicDipoleMoment = problem.properties.electronic_dipole_moment

if dipole is not None:
    nuclear_dip = dipole.nuclear_dipole_moment
    # Cập nhật moment lưỡng cực cho các thành phần x, y, z
    dipole.x_dipole.alpha += PolynomialTensor({"": nuclear_dip[0]})
    dipole.y_dipole.alpha += PolynomialTensor({"": nuclear_dip[1]})
    dipole.z_dipole.alpha += PolynomialTensor({"": nuclear_dip[2]})
    print("Đã thêm moment lưỡng cực vào Hamiltonian.")
else:
    print("Moment lưỡng cực không tồn tại trong problem.")

# Kiểm tra coefficients của Hamiltonian
coefficients = hamiltonian.electronic_integrals
#print("Coefficients của Hamiltonian:", coefficients.alpha)
#print("Các thành phần x, y, z của moment lưỡng cực:", dipole)

Đã thêm moment lưỡng cực vào Hamiltonian.


In [3]:
second_q_op = hamiltonian.second_q_op()
#print(second_q_op)

In [5]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()
qubit_p_op = mapper.map(second_q_op)
#print(qubit_p_op)

In [6]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC

ansatz = UCC(
    num_spatial_orbitals=2,
    num_particles=[1,1],
    excitations='sd',
    qubit_mapper=mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals=2,
        num_particles=[1,1],
        qubit_mapper=mapper,
    ),
    reps=1,

)

In [7]:
import numpy as np 
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA , SLSQP, L_BFGS_B, SPSA, NELDER_MEAD
from qiskit_algorithms import VQE
#from qiskit.utils import QuantumInstance

estimator = Estimator()
optimizer = SLSQP(maxiter=200)
vqe = VQE(estimator, ansatz, optimizer)
res = vqe.compute_minimum_eigenvalue(qubit_p_op)

In [8]:
from qiskit_algorithms.eigensolvers import NumPyEigensolver

numpy_solver = NumPyEigensolver()
exact_result = numpy_solver.compute_eigenvalues(qubit_p_op)
ref_value = exact_result.eigenvalues
print(f"Reference value: {ref_value  }")
print(f"VQE values: {res.optimal_value }")
print(f"Delta from reference energy value is {(res.optimal_value - ref_value)}")

Reference value: [-1.85727503]
VQE values: -1.8572750248163927
Delta from reference energy value is [5.38598788e-09]


In [ ]:
# Chuyển toán tử sang dạng ma trận
from qiskit.quantum_info import SparsePauliOp
Hopt = qubit_p_op # Hamiltonian tĩnh (static)
H_static = Hopt.to_matrix()
print(Hopt)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import Statevector, SparsePauliOp # Đảm bảo SparsePauliOp được import

# ----- GIẢ SỬ CÁC BIẾN SAU ĐÃ CÓ TỪ CÁC BƯỚC TRƯỚC -----
# num_qubits (ví dụ: 4)
# psi_0_vqe (vector trạng thái ban đầu từ VQE, là một mảng NumPy 1D)
# all_psi_t_approximated (dictionary chứa các psi(t) đã tính, ví dụ: {t1: psi_t1_array, t2: psi_t2_array, ...})

# Toán tử lưỡng cực dạng Qubit (SparsePauliOp)
# dipole_qubit_Z = ... (đã có từ mapper.map(dipole_ops["ZDipole"]))
# dipole_qubit_X = None # Hoặc SparsePauliOp nếu có
# dipole_qubit_Y = None # Hoặc SparsePauliOp nếu có

# ----- KẾT THÚC PHẦN GIẢ SỬ -----


# Hàm tính giá trị kỳ vọng (như đã định nghĩa trước)
def calculate_expectation_value(state_vector_flat, pauli_op_sparse):
    if pauli_op_sparse is None:
        return 0.0 
    if state_vector_flat is None: # Kiểm tra nếu psi(t) không hợp lệ
        return np.nan # Trả về NaN nếu trạng thái không hợp lệ
        
    sv = Statevector(state_vector_flat)
    try:
        exp_val = sv.expectation_value(pauli_op_sparse)
        return exp_val.real 
    except Exception as e_exp:
        print(f"    Lỗi khi tính giá trị kỳ vọng: {e_exp}")
        return np.nan # Trả về NaN nếu có lỗi


# Trích xuất dữ liệu thời gian và moment lưỡng cực từ kết quả mô phỏng
# Sắp xếp theo thời gian để đảm bảo đồ thị đúng thứ tự
sorted_times = sorted(all_psi_t_approximated.keys())

times_plot = []
mux_plot = []
muy_plot = []
muz_plot = []

for t_current in sorted_times:
    psi_t = all_psi_t_approximated[t_current]
    if psi_t is not None: # Chỉ xử lý nếu psi_t hợp lệ
        times_plot.append(t_current)
        
        # Tính các thành phần moment lưỡng cực tại thời điểm t_current
        # Sử dụng các biến dipole_qubit_X, dipole_qubit_Y, dipole_qubit_Z đã có
        # Nếu dipole_qubit_X hoặc Y không tồn tại, chúng sẽ trả về 0.0 từ hàm trên
        current_mux = calculate_expectation_value(psi_t, dipole_qubit_X if 'dipole_qubit_X' in locals() and dipole_qubit_X is not None else None)
        current_muy = calculate_expectation_value(psi_t, dipole_qubit_Y if 'dipole_qubit_Y' in locals() and dipole_qubit_Y is not None else None)
        current_muz = calculate_expectation_value(psi_t, dipole_qubit) # Giả định dipole_qubit_Z luôn có

        mux_plot.append(current_mux)
        muy_plot.append(current_muy)
        muz_plot.append(current_muz)
    else:
        # Nếu psi_t là None (do lỗi QSP hoặc loss cao), bạn có thể thêm NaN
        # để đồ thị bị ngắt quãng tại điểm đó, hoặc bỏ qua điểm đó.
        times_plot.append(t_current)
        mux_plot.append(np.nan)
        muy_plot.append(np.nan)
        muz_plot.append(np.nan)


# Chuyển đổi danh sách thành mảng numpy để vẽ đồ thị
times_plot_np = np.array(times_plot)
mux_plot_np = np.array(mux_plot)
muy_plot_np = np.array(muy_plot)
muz_plot_np = np.array(muz_plot)

# Vẽ đồ thị moment lưỡng cực theo thời gian
plt.figure(figsize=(12, 7))

# Chỉ vẽ nếu có dữ liệu hợp lệ (không phải toàn NaN)
if np.any(~np.isnan(mux_plot_np)) and np.any(mux_plot_np != 0): 
    plt.plot(times_plot_np, mux_plot_np, label="$\mu_x(t)$", marker='.')
if np.any(~np.isnan(muy_plot_np)) and np.any(muy_plot_np != 0): 
    plt.plot(times_plot_np, muy_plot_np, label="$\mu_y(t)$", marker='.')
if np.any(~np.isnan(muz_plot_np)): # Thành phần Z thường có giá trị
    plt.plot(times_plot_np, muz_plot_np, label="$\mu_z(t)$", marker='.')

plt.xlabel("Thời gian (fs)") # Điều chỉnh nhãn cho phù hợp
plt.ylabel("Moment lưỡng cực (a.u.)")
plt.title("Tiến hóa của Moment Lưỡng cực theo Thời gian")
plt.legend()
plt.grid(True)
plt.show()

# ----- TIẾP THEO LÀ TÍNH VÀ VẼ PHỔ HẤP THỤ (NẾU MUỐN) -----
# (Sử dụng muz_plot_np và times_plot_np cho FFT)
# (Code tính FFT và sigma_abs(omega) như ở các câu trả lời trước)

In [ ]:
# Thử trừ đi giá trị trung bình (thành phần DC)
muz_ac_signal = muz_plot_np_exact - np.mean(muz_plot_np_exact)
# Sau đó dùng muz_ac_signal cho FFT thay vì muz_plot_np_exact
# d_z_t_signal = muz_ac_signal

In [ ]:
print(f"Giá trị muz_exact_np: min={np.min(muz_plot_np_exact):.4e}, max={np.max(muz_plot_np_exact):.4e}, mean={np.mean(muz_plot_np_exact):.4e}")
print(f"Độ lớn của dao động (max-min): {np.max(muz_plot_np_exact) - np.min(muz_plot_np_exact):.4e}")

In [ ]:
import qiskit
from qiskit.quantum_info import Operator, SparsePauliOp, Statevector
from qoop.compilation.qsp import QuantumStatePreparation # Từ thư viện qoop
import numpy as np

# --- GIẢ SỬ CÁC BIẾN VÀ HÀM SAU ĐÃ ĐƯỢC ĐỊNH NGHĨA ---
# num_qubits, psi_0_vqe, ansatz_u (mạch có tham số), H_time, 
# time_dependent, time_dependent_integral, 
# combined_unique_labels (nếu dùng), num_ansatz_layers (nếu dùng)
# --- KẾT THÚC GIẢ SỬ ---

# Chọn một thời điểm cụ thể để kiểm tra, ví dụ t=0
t_to_check = 0.0

print(f"--- Kiểm tra toán tử tiến hóa tại t = {t_to_check:.4f} ---")

# 1. Tính toán toán tử tiến hóa "chính xác" U_true(t=0) từ hàm time_dependent
#    Nó phải là toán tử đơn vị (Identity)
U_true_at_t0_circuit = time_dependent(num_qubits, H_time, t_to_check)
U_true_at_t0_matrix = Operator(U_true_at_t0_circuit).data
identity_matrix = np.eye(2**num_qubits)

print(f"U_true(t=0) có gần với ma trận đơn vị không? {np.allclose(U_true_at_t0_matrix, identity_matrix)}")
if not np.allclose(U_true_at_t0_matrix, identity_matrix):
    print("CẢNH BÁO: U_true(t=0) từ hàm time_dependent không phải là ma trận đơn vị!")
    # Bạn có thể in ra U_true_at_t0_matrix để xem xét
    # print("Ma trận U_true(t=0):\n", U_true_at_t0_matrix)

# 2. Tạo target_state V(t=0)^dagger cho QSP
target_state_inverse_at_t0 = U_true_at_t0_circuit.inverse() 
# Nếu U_true(t=0) là I, thì target_state_inverse_at_t0 cũng là I.

# 3. (Tùy chọn) Tạo initial_point cho các tham số theta dựa trên t=0
initial_point_for_t0 = np.zeros(ansatz_u.num_parameters)
if hasattr(ansatz_u, 'parameters') and len(ansatz_u.parameters) > 0:
    current_H_int_at_t0 = time_dependent_integral(H_time, t=t_to_check)
    # Tại t=0, H_int(0) phải là toán tử không, nên các hệ số của nó gần 0
    # print(f"H_int(t=0) coeffs: {current_H_int_at_t0.coeffs.real}")

    label_to_coeff_map_t0 = dict(zip(current_H_int_at_t0.paulis.to_labels(), 
                                     current_H_int_at_t0.coeffs.real))
    
    for _ in range(num_ansatz_layers): 
        for label in combined_unique_labels:
            # Tại t=0, các góc xoay theta (là coeff * dt hoặc tương tự) nên gần 0
            # nếu H_int(0) gần 0.
            initial_point_for_this_t.append(label_to_coeff_map_t0.get(label, 0.0)) # Ưu tiên 0 cho t=0

    initial_point_for_t0 = np.array(initial_point_for_this_t)
    if len(initial_point_for_t0) != ansatz_u.num_parameters:
         print(f"Cảnh báo: Kích thước initial_point ({len(initial_point_for_t0)}) không khớp số tham số ansatz ({ansatz_u.num_parameters}). Sử dụng khởi tạo zeros.")
         initial_point_for_t0 = np.zeros(ansatz_u.num_parameters)

# 4. Chạy QuantumStatePreparation để tìm theta_t=0*
qsp_instance_t0 = QuantumStatePreparation(
    u=ansatz_u.copy(), 
    target_state=target_state_inverse_at_t0
)

optimal_thetas_at_t0 = None
try:
    fit_result_t0 = qsp_instance_t0.fit(
        num_steps=300, 
        metrics_func=['loss_basic'], 
        initial_point=initial_point_for_t0 if len(initial_point_for_t0) > 0 else None
    )
    
    loss_at_t0 = fit_result_t0.compiler.metrics.get('loss_basic', [1.0])[-1]
    print(f"  Loss của QSP tại t=0: {loss_at_t0:.6f} (1-loss: {1-loss_at_t0:.6f})")

    if loss_at_t0 <= 0.01: # Ngưỡng chấp nhận được
        optimal_thetas_at_t0 = qsp_instance_t0.thetas
    else:
        print(f"  Cảnh báo: Loss tại t=0 vẫn còn cao.")

except Exception as e:
    print(f"  Lỗi trong qsp.fit() tại t=0: {e}.")


# 5. Tạo U(theta_t=0*) và kiểm tra nó
if optimal_thetas_at_t0 is not None:
    U_theta_t0_optimized_circuit = ansatz_u.assign_parameters(optimal_thetas_at_t0)
    U_theta_t0_optimized_matrix = Operator(U_theta_t0_optimized_circuit).data
    
    print(f"\nU_theta_optimized(t=0) có gần với ma trận đơn vị không? {np.allclose(U_theta_t0_optimized_matrix, identity_matrix)}")
    if not np.allclose(U_theta_t0_optimized_matrix, identity_matrix):
        print("CẢNH BÁO: U_theta_optimized(t=0) (từ QSP) KHÔNG gần với ma trận đơn vị!")
        # print("Ma trận U_theta_optimized(t=0):\n", U_theta_t0_optimized_matrix)

    # 6. Tính Psi(t=0) = U(theta_t=0*) |Psi_0_vqe>
    #    Nó phải gần bằng |Psi_0_vqe>
    psi_0_vqe_col = psi_0_vqe.reshape(-1,1)
    psi_t0_approx_col = U_theta_t0_optimized_matrix @ psi_0_vqe_col
    psi_t0_approx = psi_t0_approx_col.flatten()

    print(f"\nChuẩn của Psi(t=0) xấp xỉ (dùng U_theta*): {np.linalg.norm(psi_t0_approx)}")
    # So sánh với psi_0_vqe ban đầu
    fidelity_psi0_psit0 = np.abs(psi_0_vqe.conj() @ psi_t0_approx)**2
    print(f"Fidelity giữa Psi(t=0) xấp xỉ và Psi_0_vqe ban đầu: {fidelity_psi0_psit0:.6f}")
    if not np.isclose(np.linalg.norm(psi_t0_approx), 1.0) or not np.isclose(fidelity_psi0_psit0, 1.0):
        print("CẢNH BÁO: Psi(t=0) xấp xỉ không được chuẩn hóa hoặc không khớp với Psi_0_vqe!")
else:
    print("  Không có optimal_thetas_at_t0 để kiểm tra U(theta_t=0*).")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import windows # Để sử dụng hàm cửa sổ


Gamma_val = 0.25 # Thay đổi theo giá trị thực tế của bạn
E0_val = 0.01 # Thay đổi theo giá trị thực tế của bạn
gamma_damping = 0.001

# Kiểm tra xem dữ liệu đầu vào có hợp lệ không
if len(times_plot_np_exact) < 2 or len(muz_plot_np_exact) < 2 or len(times_plot_np_exact) != len(muz_plot_np_exact):
    print("Lỗi: Dữ liệu thời gian hoặc moment lưỡng cực không đủ hoặc không khớp kích thước để thực hiện FFT.")
else:
    print(f"Số điểm dữ liệu thời gian: {len(times_plot_np_exact)}")
    print(f"Khoảng thời gian mô phỏng: từ {times_plot_np_exact[0]:.2f} đến {times_plot_np_exact[-1]:.2f}")
    
    # 1. XÁC ĐỊNH ĐƠN VỊ VÀ CHUYỂN ĐỔI NẾU CẦN
    # GIẢ SỬ: times_plot_np_exact đang ở đơn vị femtosecond (fs)
    # và E0_val, Gamma_val cũng được định nghĩa trong một hệ thống đơn vị nhất quán với Hamiltonian
    # Chúng ta sẽ chuyển thời gian sang đơn vị nguyên tử (a.u.) để nhất quán với các hằng số vật lý sau này.
    fs_to_au_time_conversion = 41.341374575751 # a.u. thời gian cho mỗi fs (1 / 0.024188843...)
    
    times_au = times_plot_np_exact / fs_to_au_time_conversion # Thời gian_au
    dt_fourier_au = times_au[1] - times_au[0]
    N_points = len(times_au)
    print(f"Bước thời gian dt (a.u.): {dt_fourier_au:.4e}")

    # 2. CHUẨN BỊ TÍN HIỆU MOMENT LƯỠNG CỰC d_z(t)
    # Áp dụng hàm cửa sổ để giảm hiệu ứng rò rỉ phổ
    window = windows.blackman(N_points) 
    d_z_t_signal = muz_plot_np_exact # Dữ liệu gốc
    d_z_t_damped = d_z_t_signal * np.exp(-gamma_damping * np.abs(times_au)) # Giả định gamma_damping là một hằng số
    d_z_t_windowed = d_z_t_damped * window

    # Tính biến đổi Fourier của d_z(t)
    d_z_omega_complex = fft(d_z_t_windowed)*dt_fourier_au # Kết quả là mảng số phức
    
    # Tạo mảng tần số góc omega cho FFT (đơn vị là Hartree nếu dt_fourier_au là a.u. thời gian)
    omega_fft_full = fftfreq(N_points, dt_fourier_au) * 2 * np.pi

    # 3. CHUẨN BỊ TÍN HIỆU TRƯỜNG ĐIỆN TỪ E_z(t) VÀ TÍNH FFT
    # Hàm f(t) của bạn (trường Lorentz)
    def f_t_lorentz(t_values_au, E0, Gamma_au): # Các tham số nên ở đơn vị a.u.
        # Nếu Gamma_val của bạn từ H_time là fs, cần chuyển nó sang a.u.
        # Gamma_au = Gamma_val / fs_to_au_time_conversion (NẾU Gamma_val là fs)
        # Ở đây giả sử Gamma_val đã là a.u. hoặc bạn đã chuyển đổi nó
        return (E0 / np.pi) * Gamma_au / (Gamma_au**2 + t_values_au**2)

    # Giả sử Gamma_val bạn có từ H_time là fs, chuyển sang a.u.
    # Nếu Gamma_val đã là a.u. thì không cần dòng này.
    Gamma_au_val = Gamma_val / fs_to_au_time_conversion 
    
    electric_field_z_t = f_t_lorentz(times_au, E0_val, Gamma_au_val)
    E_z_t_damped = electric_field_z_t * np.exp(-gamma_damping * np.abs(times_au)) # Giả định gamma_damping là một hằng số
    E_z_t_windowed = E_z_t_damped * window 
    
    E_z_omega_complex = fft(E_z_t_windowed)*dt_fourier_au # Kết quả là mảng số phức

    # In một vài giá trị để kiểm tra
    # print("d_z_t_signal (first 5):", d_z_t_signal[:5])
    # print("electric_field_z_t (first 5):", electric_field_z_t[:5])
    # print("d_z_omega_complex (first 5):", d_z_omega_complex[:5])
    # print("E_z_omega_complex (first 5):", E_z_omega_complex[:5])
    # print("omega_fft_full (first 5):", omega_fft_full[:5])


    # 4. TÍNH ALPHA_ZZ(OMEGA)
    # Dịch chuyển 0-frequency component về giữa mảng
    d_z_omega_shifted = fftshift(d_z_omega_complex)
    E_z_omega_shifted = fftshift(E_z_omega_complex)
    omega_fft_shifted = fftshift(omega_fft_full)

    # Chỉ lấy các tần số dương (omega >= 0)
    positive_freq_mask = omega_fft_shifted >= 0 
    
    omega_positive = omega_fft_shifted[positive_freq_mask]
    d_z_omega_positive = d_z_omega_shifted[positive_freq_mask]
    E_z_omega_positive = E_z_omega_shifted[positive_freq_mask]
    
    #alpha_zz_omega = np.zeros_like(d_z_omega_positive, dtype=complex)
    # Ngưỡng nhỏ để tránh chia cho các giá trị E_z_omega rất gần 0
    # Nên kiểm tra xem E_z_omega_positive có thực sự nhỏ ở đâu đó không
    epsilon = 1e-12 # Một số rất nhỏ để tránh chia cho 0
    alpha_zz_omega = d_z_omega_positive / (E_z_omega_positive + epsilon)
    
    # print("omega_positive (first 5, Hartree):", omega_positive[:5])
    # print("alpha_zz_omega (first 5):", alpha_zz_omega[:5])

    # 5. TÍNH SIGMA_ABS(OMEGA)
    c_au = 137.035999084 

    # ...
    # imag_alpha_for_absorption = np.imag(alpha_zz_omega) # alpha_zz_omega từ d_Z / (E_Z + epsilon)
    #
    # # Đảo dấu Im[alpha] nếu nó luôn âm/không dương và bạn kỳ vọng hấp thụ dương
    # # Dựa trên kết quả Min Im[alpha_zz]: -9.98e+02, Max Im[alpha_zz]: -0.00e+00
    # # thì việc đảo dấu là cần thiết.
    corrected_imag_alpha = -np.imag(alpha_zz_omega) 

    sigma_abs_omega_values = (4 * np.pi * omega_positive / c_au) * corrected_imag_alpha

    # Loại bỏ các giá trị âm rất nhỏ có thể còn sót lại do sai số số học
    sigma_abs_omega_values[sigma_abs_omega_values < 1e-9] = 0 

    # ... (phần vẽ đồ thị sigma_abs_omega_values theo omega_ev_plot) ...


    # 6. CHUYỂN ĐỔI OMEGA SANG EV ĐỂ VẼ ĐỒ THỊ
    hartree_to_ev = 27.211386245988
    omega_ev_plot = omega_positive * hartree_to_ev

    # 7. VẼ PHỔ HẤP THỤ
    plt.figure(figsize=(10, 6))
    plt.plot(omega_ev_plot, sigma_abs_omega_values) # Sử dụng sigma_abs_omega_values mới
    plt.xlabel("Năng lượng $\omega$ (eV)")
    plt.ylabel("$\sigma_{abs}(\omega)$ (a.u.)")
    plt.title("Phổ hấp thụ (đã điều chỉnh dấu Im[alpha])")
    plt.grid(True)

    # Điều chỉnh giới hạn trục x và y nếu cần để thấy rõ các đỉnh
    if np.any(sigma_abs_omega_values > 1e-9): 
        meaningful_indices = sigma_abs_omega_values > (0.01 * np.max(sigma_abs_omega_values[~np.isnan(sigma_abs_omega_values)]))
        if np.any(meaningful_indices):
            min_ev = np.min(omega_ev_plot[meaningful_indices])
            max_ev = np.max(omega_ev_plot[meaningful_indices])
            padding_ev = (max_ev - min_ev) * 0.1 if (max_ev - min_ev) > 0 else 1.0
            plt.xlim(max(0, min_ev - padding_ev), max_ev + padding_ev)
        else: 
            relevant_omega_indices = omega_ev_plot <= 20 
            if np.any(relevant_omega_indices) and len(omega_ev_plot[relevant_omega_indices]) > 0 : # Thêm kiểm tra len
                plt.xlim(0, omega_ev_plot[relevant_omega_indices][-1])
            else:
                plt.xlim(0,1) 
    else: 
        plt.xlim(0, 20) 

    plt.ylim(bottom=0) 
    plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(omega_positive * hartree_to_ev, np.real(alpha_zz_omega))
plt.title("Re[$\\alpha_{zz}(\omega)$]")
plt.xlabel("Năng lượng $\omega$ (eV)")
plt.ylabel("Phần thực")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(omega_positive * hartree_to_ev, np.imag(alpha_zz_omega))
plt.title("Im[$\\alpha_{zz}(\omega)$]")
plt.xlabel("Năng lượng $\omega$ (eV)")
plt.ylabel("Phần ảo")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(omega_fft_shifted * hartree_to_ev, np.abs(d_z_omega_shifted))
plt.title("$|d_Z(\omega)|$")
plt.xlabel("Năng lượng $\omega$ (eV)")
plt.ylabel("Biên độ")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(omega_fft_shifted * hartree_to_ev, np.abs(E_z_omega_shifted))
plt.title("$|E_Z(\omega)|$")
plt.xlabel("Năng lượng $\omega$ (eV)")
plt.ylabel("Biên độ")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ... sau khi tính corrected_imag_alpha ...
print(f"Giá trị lớn nhất của corrected_imag_alpha (-Im[alpha_zz]): {np.max(corrected_imag_alpha):.4e}")
print(f"Giá trị nhỏ nhất của corrected_imag_alpha (-Im[alpha_zz]): {np.min(corrected_imag_alpha):.4e}")
# Cũng nên xem xét một vài giá trị ở vùng có đỉnh
# if len(corrected_imag_alpha) > 0:
#     peak_region_sample = corrected_imag_alpha[len(corrected_imag_alpha)//4 : 3*len(corrected_imag_alpha)//4] # Ví dụ một vùng
#     if len(peak_region_sample) > 0:
#         print(f"Max corrected_imag_alpha trong vùng giữa phổ: {np.max(peak_region_sample):.4e}")

In [ ]:
print(f"Min Im[alpha_zz]: {np.min(np.imag(alpha_zz_omega)):.2e}, Max Im[alpha_zz]: {np.max(np.imag(alpha_zz_omega)):.2e}")
print(f"Min sigma_abs: {np.min(sigma_abs_omega_values):.2e}, Max sigma_abs: {np.max(sigma_abs_omega_values):.2e}")

In [ ]:
if np.any(sigma_abs_omega_values > 1e-9): # Chỉ giới hạn nếu có tín hiệu
    meaningful_indices = sigma_abs_omega_values > (0.01 * np.max(sigma_abs_omega_values[~np.isnan(sigma_abs_omega_values)]))
    # ...
else: # Nếu tín hiệu rất yếu, đặt giới hạn mặc định
    plt.xlim(0, 20)

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp # Sử dụng numpy của Pennylane
import numpy as np # Numpy gốc
import matplotlib.pyplot as plt
from qiskit.quantum_info import random_unitary # Để tạo target ngẫu nhiên

# --- Hàm ansatz Pennylane (đã định nghĩa ở lần trước) ---
def pennylane_ansatz_from_qiskit_pauli_evo(thetas, num_qubits, pauli_labels_fixed, num_layers_ansatz):
    # ... (Nội dung hàm như đã cung cấp ở lần trước) ...
    if not pauli_labels_fixed:
        raise ValueError("pauli_labels_fixed không được rỗng.")
    num_params_per_layer = len(pauli_labels_fixed)
    num_params_expected = num_params_per_layer * num_layers_ansatz
    if len(thetas) != num_params_expected:
        raise ValueError(
            f"Số lượng tham số `thetas` không khớp. Mong đợi {num_params_expected}, nhận được {len(thetas)}."
        )
    param_idx = 0
    for _ in range(num_layers_ansatz):
        for pauli_qiskit_str in pauli_labels_fixed:
            if len(pauli_qiskit_str) != num_qubits:
                raise ValueError(
                    f"Chuỗi Pauli '{pauli_qiskit_str}' có độ dài {len(pauli_qiskit_str)}, "
                    f"không khớp với num_qubits={num_qubits}."
                )
            pennylane_observable = None
            if all(c == 'I' for c in pauli_qiskit_str):
                if num_qubits > 0:
                    pennylane_observable = qml.Identity(0)
                    for i_w in range(1, num_qubits):
                        pennylane_observable @= qml.Identity(i_w)
                elif num_qubits == 0:
                     pennylane_observable = qml.Identity(0)
                else:
                    raise ValueError("Số qubit phải không âm.")
            else:
                try:
                    pennylane_observable = qml.pauli.string_to_pauli_word(pauli_qiskit_str)
                except AttributeError:
                    print(f"Warning: qml.pauli.string_to_pauli_word không thành công cho '{pauli_qiskit_str}'. Thử xây dựng thủ công.")
                    ops_list = []
                    for i_wire in range(num_qubits):
                        char = pauli_qiskit_str[i_wire]
                        if char == 'X': ops_list.append(qml.PauliX(i_wire))
                        elif char == 'Y': ops_list.append(qml.PauliY(i_wire))
                        elif char == 'Z': ops_list.append(qml.PauliZ(i_wire))
                        elif char == 'I': pass
                        else: raise ValueError(f"Ký tự Pauli không hợp lệ '{char}' trong chuỗi '{pauli_qiskit_str}'")
                    if not ops_list:
                        raise ValueError(f"Không thể tạo observable từ chuỗi Pauli '{pauli_qiskit_str}' (ops_list rỗng sau khi lọc I).")
                    else:
                        pennylane_observable = qml.prod(*ops_list)
            if pennylane_observable is not None:
                 qml.exp(pennylane_observable, -1j * thetas[param_idx])
            param_idx += 1
# --- Kết thúc hàm ansatz ---

# --- Các tham số thực tế từ bài toán Qiskit của bạn ---
num_qubits_actual = 4
pauli_labels_actual = combined_unique_labels  # Sử dụng danh sách đã tạo từ Hopt và dipole_qubit
num_layers_actual = 1
num_thetas_actual = len(pauli_labels_actual) * num_layers_actual

# Khởi tạo thetas
if num_thetas_actual > 0:
    initial_theta_values_np = np.random.uniform(0, 0.1, num_thetas_actual)
    thetas = pnp.array(initial_theta_values_np, requires_grad=True)
else:
    thetas = pnp.array([], requires_grad=False)

# Tạo target unitary CỐ ĐỊNH (dưới dạng ma trận NumPy)
time_pen_sim = np.linspace(0, 10, 4) 

#Danh sách lưu trữ
all_optimized_U_theta_t = {}
all_psi_t_pen = {}
all_costs_over_time = {}    

for time_pen in time_pen_sim:
    print(f"\nBắt đầu tối ưu hóa cho time_pen = {time_pen}...")

    target_pen = time_dependent(N, H_time, time_pen)
    target_pen_matrix = None
    if num_qubits_actual > 0:
        try:
            target_pen_matrix = Operator(target_pen).data   # Chuyển thành ma trận NumPy
        except Exception as e:
            print(f"  Lỗi khi tạo target matrix cho t={time_pen:.4f}: {e}")
            continue
    elif num_qubits_actual == 0:
        current_target_matrix_np = np.array([[1.0+0.j]])
        print(f"  Đã tạo target U_true(t={time_pen:.4f}) cho 0 qubit.")
    else:
        print(f"  Số qubit không hợp lệ: {num_qubits_actual}")
        continue


# Hàm cost để truyền vào optimizer, SỬ DỤNG qml.matrix NHƯ MỘT TRANSFORM
def cost_for_pennylane_optimization(thetas_input):
    matrix = qml.matrix(pennylane_ansatz_from_qiskit_pauli_evo,
                               wire_order=range(num_qubits_actual)
                              )(thetas_input, num_qubits_actual, pauli_labels_actual, num_layers_actual)
    return cost_func.c_hst(matrix, target_pen_matrix)

# --- Vòng lặp tối ưu hóa (tương tự "baseline.ipynb") ---
if num_thetas_actual > 0:
    steps = 100
    costs = []
    opt = qml.AdamOptimizer(stepsize=0.01)
    current_thetas_for_opt = pnp.copy(thetas, requires_grad=True) # Sử dụng thetas đã khởi tạo

    print(f"\nBắt đầu tối ưu hóa ansatz Pennylane với {num_thetas_actual} tham số...")
    for n in range(steps):
        current_thetas_for_opt, prev_cost = opt.step_and_cost(cost_for_pennylane_optimization, current_thetas_for_opt)
        costs.append(prev_cost.numpy() if hasattr(prev_cost, 'numpy') else prev_cost) # Lưu lại prev_cost
        

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp # Sử dụng numpy của Pennylane cho các tham số có đạo hàm

def pennylane_ansatz_from_qiskit_pauli_evo(thetas, num_qubits, pauli_labels_fixed, num_layers_ansatz):
    """
    Tạo một ansatz Pennylane tương đương với việc áp dụng một chuỗi các PauliEvolutionGate của Qiskit.
    Cấu trúc của Qiskit ansatz: U(theta) = product_L (product_k exp(-i * theta_Lk * P_k))
    Trong Pennylane, qml.exp(op, coeff) thực hiện exp(coeff * op).
    Chúng ta muốn thực hiện exp(-i * theta_Lk * P_k), vậy:
        op = P_k (dưới dạng PauliWord của Pennylane)
        coeff = -1j * theta_Lk

    Args:
        thetas (pnp.ndarray): Mảng 1D các tham số theta, được làm phẳng (flattened).
                              Thứ tự phải khớp với cách Qiskit ansatz sử dụng chúng
                              (tức là, tất cả các theta cho lớp 1, sau đó tất cả cho lớp 2, v.v.).
        num_qubits (int): Số qubit của hệ thống.
        pauli_labels_fixed (list[str]): Danh sách các chuỗi Pauli CỐ ĐỊNH (P_k)
                                      dưới dạng chuỗi ký tự kiểu Qiskit (ví dụ: "IXYZ", "ZZI").
                                      Độ dài mỗi chuỗi phải bằng num_qubits.
        num_layers_ansatz (int): Số lớp (N_L) của ansatz.
    """
    if not pauli_labels_fixed:
        raise ValueError("pauli_labels_fixed không được rỗng.")
    
    num_params_per_layer = len(pauli_labels_fixed)
    num_params_expected = num_params_per_layer * num_layers_ansatz

    if len(thetas) != num_params_expected:
        raise ValueError(
            f"Số lượng tham số `thetas` không khớp. Mong đợi {num_params_expected}, nhận được {len(thetas)}."
        )

    param_idx = 0
    for _ in range(num_layers_ansatz):
        for pauli_qiskit_str in pauli_labels_fixed:
            if len(pauli_qiskit_str) != num_qubits:
                raise ValueError(
                    f"Chuỗi Pauli '{pauli_qiskit_str}' có độ dài {len(pauli_qiskit_str)}, "
                    f"không khớp với num_qubits={num_qubits}."
                )
            
            # Chuyển đổi chuỗi Pauli Qiskit thành đối tượng PauliWord của Pennylane
            # Pennylane's PauliWord.from_paulistring chấp nhận định dạng chuỗi tương tự Qiskit
            # và tự động ánh xạ tới các wires 0, 1, ..., num_qubits-1
            try:
                pennylane_observable = qml.pauli.PauliWord.from_paulistring(pauli_qiskit_str)
            except Exception as e:
                # qml.pauli.PauliWord.from_paulistring có thể không tồn tại trong mọi phiên bản
                # Hoặc có thể cần qml.pauli.string_to_pauli_word(pauli_qiskit_str)
                # Thay thế bằng cách xây dựng thủ công nếu cần:
                ops_list = []
                for i_wire in range(num_qubits):
                    char = pauli_qiskit_str[i_wire]
                    if char == 'X': ops_list.append(qml.PauliX(i_wire))
                    elif char == 'Y': ops_list.append(qml.PauliY(i_wire))
                    elif char == 'Z': ops_list.append(qml.PauliZ(i_wire))
                    elif char == 'I': ops_list.append(qml.Identity(i_wire))
                    else: raise ValueError(f"Ký tự Pauli không hợp lệ '{char}' trong chuỗi '{pauli_qiskit_str}'")
                
                if not ops_list: # Trường hợp num_qubits = 0
                    pennylane_observable = qml.Identity(0) # Hoặc xử lý khác
                else:
                    pennylane_observable = ops_list[0]
                    for i_op in range(1, len(ops_list)):
                        pennylane_observable = pennylane_observable @ ops_list[i_op]
                # print(f"Đã tạo observable thủ công cho: {pauli_qiskit_str} -> {pennylane_observable}")


            # Áp dụng toán tử e^(-i * theta * P_k)
            # Lưu ý: Pennylane qml.exp(op, coeff) là exp(coeff * op)
            # coeff = -1j * theta
            qml.exp(pennylane_observable, -1j * thetas[param_idx])
            
            param_idx += 1
        
        # Trong Pennylane, không có khái niệm "barrier" ảnh hưởng đến ma trận unitary
        # khi sử dụng qml.matrix hoặc trong QNode.
        # qml.Barrier có thể được thêm vào QNode nếu bạn muốn nó hiển thị khi vẽ mạch,
        # nhưng nó sẽ không thay đổi kết quả tính toán ma trận.
        
# --- Ví dụ sử dụng ---
# Định nghĩa các tham số cho ví dụ
num_qubits_example = 4
# Sửa lại các chuỗi Pauli để có độ dài 4
pauli_labels_example = ["IXZI", "ZYII", "IIII"] # Ví dụ: thêm 'I' vào cuối
# Hoặc bạn có thể định nghĩa lại chúng cho phù hợp với hệ 4 qubit, ví dụ:
# pauli_labels_example = ["XXII", "IZZI", "ZIZI"]

num_layers_example = 1
num_thetas_example = len(pauli_labels_example) * num_layers_example

# Tạo các tham số theta ngẫu nhiên cho Pennylane (cần requires_grad=True nếu muốn tối ưu)
thetas_example = pnp.random.uniform(0, 2 * np.pi, num_thetas_example, requires_grad=True)

# Định nghĩa một device của Pennylane
dev_example = qml.device("default.qubit", wires=num_qubits_example)

@qml.qnode(dev_example)
def simple_test_circuit(params):
    qml.RX(params[0], wires=0)
    if num_qubits_example > 1:
        qml.RY(params[1], wires=1)
        qml.CNOT(wires=[0,1])
    return qml.matrix(wires=range(num_qubits_example))

# Tạo a.thetas giả cho test
num_simple_params = 2 if num_qubits_example > 1 else 1
if num_qubits_example == 0: num_simple_params = 0

if num_simple_params > 0:
    test_params = pnp.array(np.random.rand(num_simple_params), requires_grad=True)
    try:
        mat_rep_test = simple_test_circuit(test_params)
        print("Ma trận từ simple_test_circuit:\n", mat_rep_test)
    except Exception as e:
        print("Lỗi với simple_test_circuit:", e)
else:
    # Test cho 0 qubit (nếu có)
    # @qml.qnode(qml.device("default.qubit", wires=1)) # Cần ít nhất 1 wire cho device
    # def empty_circuit():
    #     return qml.matrix(wires=[0]) # Hoặc qml.state()
    # print(empty_circuit())
    print("Không có tham số cho simple_test_circuit vì num_qubits_example = 0")

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp # Sử dụng numpy của Pennylane
import numpy as np # Numpy gốc
import matplotlib.pyplot as plt
from qiskit.quantum_info import random_unitary # Để tạo target ngẫu nhiên

# --- Hàm ansatz Pennylane (đã định nghĩa ở lần trước) ---
def pennylane_ansatz_from_qiskit_pauli_evo(thetas, num_qubits, pauli_labels_fixed, num_layers_ansatz):
    # ... (Nội dung hàm như đã cung cấp ở lần trước) ...
    if not pauli_labels_fixed:
        raise ValueError("pauli_labels_fixed không được rỗng.")
    num_params_per_layer = len(pauli_labels_fixed)
    num_params_expected = num_params_per_layer * num_layers_ansatz
    if len(thetas) != num_params_expected:
        raise ValueError(
            f"Số lượng tham số `thetas` không khớp. Mong đợi {num_params_expected}, nhận được {len(thetas)}."
        )
    param_idx = 0
    for _ in range(num_layers_ansatz):
        for pauli_qiskit_str in pauli_labels_fixed:
            if len(pauli_qiskit_str) != num_qubits:
                raise ValueError(
                    f"Chuỗi Pauli '{pauli_qiskit_str}' có độ dài {len(pauli_qiskit_str)}, "
                    f"không khớp với num_qubits={num_qubits}."
                )
            pennylane_observable = None
            if all(c == 'I' for c in pauli_qiskit_str):
                if num_qubits > 0:
                    pennylane_observable = qml.Identity(0)
                    for i_w in range(1, num_qubits):
                        pennylane_observable @= qml.Identity(i_w)
                elif num_qubits == 0:
                     pennylane_observable = qml.Identity(0)
                else:
                    raise ValueError("Số qubit phải không âm.")
            else:
                try:
                    pennylane_observable = qml.pauli.string_to_pauli_word(pauli_qiskit_str)
                except AttributeError:
                    print(f"Warning: qml.pauli.string_to_pauli_word không thành công cho '{pauli_qiskit_str}'. Thử xây dựng thủ công.")
                    ops_list = []
                    for i_wire in range(num_qubits):
                        char = pauli_qiskit_str[i_wire]
                        if char == 'X': ops_list.append(qml.PauliX(i_wire))
                        elif char == 'Y': ops_list.append(qml.PauliY(i_wire))
                        elif char == 'Z': ops_list.append(qml.PauliZ(i_wire))
                        elif char == 'I': pass
                        else: raise ValueError(f"Ký tự Pauli không hợp lệ '{char}' trong chuỗi '{pauli_qiskit_str}'")
                    if not ops_list:
                        raise ValueError(f"Không thể tạo observable từ chuỗi Pauli '{pauli_qiskit_str}' (ops_list rỗng sau khi lọc I).")
                    else:
                        pennylane_observable = qml.prod(*ops_list)
            if pennylane_observable is not None:
                 qml.exp(pennylane_observable, -1j * thetas[param_idx])
            param_idx += 1
# --- Kết thúc hàm ansatz ---

# --- Các tham số thực tế từ bài toán Qiskit của bạn ---
num_qubits_actual = 4
pauli_labels_actual = combined_unique_labels  # Sử dụng danh sách đã tạo từ Hopt và dipole_qubit
num_layers_actual = 1
num_thetas_actual = len(pauli_labels_actual) * num_layers_actual

# Khởi tạo thetas
if num_thetas_actual > 0:
    initial_theta_values_np = np.random.uniform(0, 0.1, num_thetas_actual)
    thetas_evolving_pen = pnp.array(initial_theta_values_np, requires_grad=True)
else:
    thetas_evolving_pen = pnp.array([], requires_grad=False)

# Tạo target unitary CỐ ĐỊNH (dưới dạng ma trận NumPy)
time_pen_sim = np.linspace(0, 10, 4) 

#Danh sách lưu trữ
all_optimized_U_theta_t = {}
all_psi_t_pen = {}
all_costs_over_time = {}    

# Khởi tạo optimizer
steps = 100
opt = qml.AdamOptimizer(stepsize=0.01)

for time_pen in time_pen_sim:
    print(f"\nBắt đầu tối ưu hóa cho time_pen = {time_pen:.4f}...")

    target_pen_circuit = time_dependent(N, H_time, time_pen)
    target_pen_matrix = None
    if num_qubits_actual > 0:
        try:
            target_pen_matrix = Operator(target_pen_circuit).data   # Chuyển thành ma trận NumPy
        except Exception as e:
            print(f"  Lỗi khi tạo target matrix cho t={time_pen:.4f}: {e}")
            continue
    elif num_qubits_actual == 0:
        target_pen_matrix = np.array([[1.0+0.j]])
        print(f"  Đã tạo target U_true(t={time_pen:.4f}) cho 0 qubit.")
    else:
        print(f"  Số qubit không hợp lệ: {num_qubits_actual}")
        continue


# Hàm cost để truyền vào optimizer, SỬ DỤNG qml.matrix NHƯ MỘT TRANSFORM
def cost_for_this_specific_t(thetas_input):
    if num_thetas_actual == 0:
            ansatz_matrix_no_params = np.eye(2**num_qubits_actual if num_qubits_actual > 0 else 1, dtype=complex)
            return cost_func.c_hst(ansatz_matrix_no_params, target_pen_matrix)
    
    matrix = qml.matrix(pennylane_ansatz_from_qiskit_pauli_evo,
                               wire_order=range(num_qubits_actual)
                              )(thetas_input, num_qubits_actual, pauli_labels_actual, num_layers_actual)
    return cost_func.c_hst(matrix, target_pen_matrix)

#current_thetas_for_opt = pnp.copy(thetas_evolving_pen, requires_grad=True)
costs = []

# --- Vòng lặp tối ưu hóa (tương tự "baseline.ipynb") ---
if num_thetas_actual > 0:
    print(f" Bắt đầu tối ưu hóa cho t = {time_pen:.4f} với {num_thetas_actual} tham số...")
    for n in range(steps):
        thetas_evolving_pen, prev_cost = opt.step_and_cost(cost_for_this_specific_t, thetas_evolving_pen)
        #current_thetas_for_opt, prev_cost = opt.step_and_cost(cost_for_pennylane_optimization_t, current_thetas_for_opt)
        # costs.append(prev_cost.numpy() if hasattr(prev_cost, 'numpy') else prev_cost) # Lưu lại prev_cost
        # if (n + 1) % (steps_per_time_point // 2 if steps_per_time_point >=2 else 1) == 0: # In thường xuyên hơn
        #             print(f"    t={t_current_sim:.2f}, Opt. step {n+1}/{steps_per_time_point}, Cost: {prev_cost:.6f}")
        # if prev_cost < constant_error:
        #             print(f"    Đạt ngưỡng lỗi ở bước {n+1} cho t={t_current_sim:.2f}")
        #             break
    #thetas = current_thetas_for_opt

all_costs_over_time[time_pen] = costs
optimized_thetas_val = thetas_evolving_pen.numpy() if hasattr(thetas_evolving_pen, 'numpy') else thetas_evolving_pen

U_theta_op_val = None
if num_thetas_actual > 0:
    U_theta_op_val = qml.matrix(pennylane_ansatz_from_qiskit_pauli_evo,
                               wire_order=range(num_qubits_actual)
                              )(optimized_thetas_val, num_qubits_actual, pauli_labels_actual, num_layers_actual)

all_optimized_U_theta_t[time_pen] = U_theta_op_val.numpy() if hasattr(U_theta_op_val, 'numpy') else U_theta_op_val


psi_0_vqe_col = psi_0_vqe.reshape(-1, 1) 
U_mat_np_for_psi = all_optimized_U_theta_t[time_pen]

psi_t_col_approx = U_mat_np_for_psi @ psi_0_vqe_col
psi_t_approx = psi_t_col_approx.flatten()
all_psi_t_pen[time_pen] = psi_t_approx
        

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp # Sử dụng numpy của Pennylane cho các tham số có đạo hàm
import numpy as np # Sử dụng numpy gốc cho một số tiện ích nếu cần
import matplotlib.pyplot as plt # Để vẽ đồ thị nếu cần

def pennylane_ansatz_from_qiskit_pauli_evo(thetas, num_qubits, pauli_labels_fixed, num_layers_ansatz):
    """
    Tạo một ansatz Pennylane tương đương với việc áp dụng một chuỗi các PauliEvolutionGate của Qiskit.
    Cấu trúc của Qiskit ansatz: U(theta) = product_L (product_k exp(-i * theta_Lk * P_k))
    Trong Pennylane, qml.exp(op, coeff) thực hiện exp(coeff * op).
    Chúng ta muốn thực hiện exp(-i * theta_Lk * P_k), vậy:
        op = P_k (dưới dạng Observable của Pennylane)
        coeff = -1j * theta_Lk

    Args:
        thetas (pnp.ndarray): Mảng 1D các tham số theta, được làm phẳng (flattened).
                              Thứ tự phải khớp với cách Qiskit ansatz sử dụng chúng.
        num_qubits (int): Số qubit của hệ thống.
        pauli_labels_fixed (list[str]): Danh sách các chuỗi Pauli CỐ ĐỊNH (P_k)
                                      dưới dạng chuỗi ký tự kiểu Qiskit (ví dụ: "IXYZ", "ZZI").
                                      Độ dài mỗi chuỗi phải bằng num_qubits.
        num_layers_ansatz (int): Số lớp (N_L) của ansatz.
    """
    if not pauli_labels_fixed:
        raise ValueError("pauli_labels_fixed không được rỗng.")
    
    num_params_per_layer = len(pauli_labels_fixed)
    num_params_expected = num_params_per_layer * num_layers_ansatz

    if len(thetas) != num_params_expected:
        raise ValueError(
            f"Số lượng tham số `thetas` không khớp. Mong đợi {num_params_expected}, nhận được {len(thetas)}."
        )

    param_idx = 0
    for _ in range(num_layers_ansatz):
        for pauli_qiskit_str in pauli_labels_fixed:
            if len(pauli_qiskit_str) != num_qubits:
                raise ValueError(
                    f"Chuỗi Pauli '{pauli_qiskit_str}' có độ dài {len(pauli_qiskit_str)}, "
                    f"không khớp với num_qubits={num_qubits}."
                )
            
            pennylane_observable = None
            # Chuyển đổi chuỗi Pauli Qiskit thành đối tượng Observable của Pennylane
            if all(c == 'I' for c in pauli_qiskit_str): # Xử lý trường hợp toàn Identity
                if num_qubits > 0:
                    pennylane_observable = qml.Identity(0)
                    for i_w in range(1, num_qubits):
                        pennylane_observable @= qml.Identity(i_w)
                elif num_qubits == 0:
                     pennylane_observable = qml.Identity(0) 
                else:
                    raise ValueError("Số qubit phải không âm.")
            else:
                try:
                    # Sử dụng qml.pauli.string_to_pauli_word cho Pennylane 0.38.0
                    pennylane_observable = qml.pauli.string_to_pauli_word(pauli_qiskit_str)
                except AttributeError: 
                    # Fallback này có thể không cần thiết nếu string_to_pauli_word luôn hoạt động
                    # nhưng giữ lại để phòng trường hợp phiên bản rất cũ hoặc lỗi import
                    print(f"Warning: qml.pauli.string_to_pauli_word không thành công cho '{pauli_qiskit_str}'. Thử xây dựng thủ công.")
                    ops_list = []
                    # is_identity_term đã được xử lý ở trên, nên ở đây pauli_qiskit_str không phải toàn 'I'
                    for i_wire in range(num_qubits):
                        char = pauli_qiskit_str[i_wire]
                        if char == 'X': ops_list.append(qml.PauliX(i_wire))
                        elif char == 'Y': ops_list.append(qml.PauliY(i_wire))
                        elif char == 'Z': ops_list.append(qml.PauliZ(i_wire))
                        elif char == 'I': pass 
                        else: raise ValueError(f"Ký tự Pauli không hợp lệ '{char}' trong chuỗi '{pauli_qiskit_str}'")
                    
                    if not ops_list: # Nếu sau khi bỏ 'I' mà ops_list rỗng (ví dụ chuỗi "I" cho 1 qubit)
                                     # điều này không nên xảy ra vì trường hợp all('I') đã được xử lý.
                                     # Tuy nhiên, nếu num_qubits > 0 và ops_list rỗng ở đây, đó là lỗi.
                        raise ValueError(f"Không thể tạo observable từ chuỗi Pauli '{pauli_qiskit_str}' (ops_list rỗng sau khi lọc I).")
                    else:
                        pennylane_observable = qml.prod(*ops_list) # Sửa lỗi NameError ở đây

            # Áp dụng toán tử e^(-i * theta * P_k)
            if pennylane_observable is not None:
                 qml.exp(pennylane_observable, -1j * thetas[param_idx])
            param_idx += 1

# --- Sử dụng với các tham số thực tế từ bài toán của bạn ---
# THAY THẾ CÁC GIÁ TRỊ NÀY BẰNG GIÁ TRỊ THỰC TẾ TỪ CODE QISKIT CỦA BẠN
num_qubits_actual = 4  # Từ Hopt.num_qubits trong code Qiskit của bạn
pauli_labels_actual = combined_unique_labels
# ^^^ QUAN TRỌNG: Đây chỉ là VÍ DỤ. 
# Hãy thay thế bằng danh sách `combined_unique_labels` thực tế của bạn.
# Đảm bảo mỗi chuỗi có độ dài bằng num_qubits_actual.

num_layers_actual = 1  # Từ num_ansatz_layers trong code Qiskit của bạn

num_thetas_actual = len(pauli_labels_actual) * num_layers_actual

# Tạo các tham số theta ban đầu cho Pennylane
if num_thetas_actual > 0:
    initial_theta_values_np_actual = np.random.uniform(0, 0.1, num_thetas_actual) # Khởi tạo nhỏ
    thetas_actual = pnp.array(initial_theta_values_np_actual, requires_grad=True)
else:
    thetas_actual = pnp.array([], requires_grad=False)



In [ ]:
import numpy as np
from qiskit.quantum_info import Operator 

t1 = times_for_qsp[0]  
t2 = times_for_qsp[1] 
t3 = times_for_qsp[2] 
t4 = times_for_qsp[3]

target_unitary_t1_circuit = time_dependent(num_qubits, H_time, t1)
U_t1_matrix = Operator(target_unitary_t1_circuit).data

target_unitary_t2_circuit = time_dependent(num_qubits, H_time, t2)
U_t2_matrix = Operator(target_unitary_t2_circuit).data

target_unitary_t3_circuit = time_dependent(num_qubits,H_time,t3)
U_t3_matrix = Operator(target_unitary_t3_circuit).data

target_unitary_t4_circuit = time_dependent(num_qubits, H_time,t4)
U_t4_matrix = Operator(target_unitary_t4_circuit).data

difference_matrix1 = U_t1_matrix - U_t2_matrix
difference_matrix2 = U_t3_matrix - U_t4_matrix

norm_of_difference1 = np.linalg.norm(difference_matrix1, 'fro')
norm_of_difference2 = np.linalg.norm(difference_matrix2, 'fro')

print(f"Toán tử U(t1) tại t1 = {t1}:\n{U_t1_matrix}")
print(f"Toán tử U(t2) tại t2 = {t2}:\n{U_t2_matrix}")
print(f"Toán tử U(t3) tại t3 = {t3}:\n{U_t3_matrix}")
print(f"Toán tử U(t4) tại t4 = {t4}:\n{U_t4_matrix}")
#print(f"Norm Frobenius của sự khác biệt giữa U(t1) và U(t2): {norm_of_difference1}")
#print(f"Norm Frobenius của sự khác biệt giữa U(t3) và U(t44): {norm_of_difference2}")